In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score

In [2]:
# Load data
url = 'https://raw.githubusercontent.com/beespinosa1/Inter/main/Escenario1.csv'
df = pd.read_csv(url)
df = pd.read_csv(url, low_memory=False)

In [3]:
# Normalize the data
scaler = MinMaxScaler()
data_to_process = df.iloc[:, 4:-1]
x_normalized = pd.DataFrame(scaler.fit_transform(data_to_process), columns=data_to_process.columns)

In [4]:
# Split into train and test sets
y = df['label']
label_map = {'AUDIO': 0, 'BROWSING': 1, 'CHAT': 2, 'FILE-TRANSFER': 3, 'MAIL': 4, 'P2P': 5, 'VIDEO': 6, 'VOIP': 7}
y_normalized = y.apply(lambda x: label_map[x] if x in label_map else -1)
x_train, x_test, y_train, y_test = train_test_split(x_normalized, y_normalized, test_size=0.3, random_state=0)

In [6]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


In [7]:
# Define the Stacked Autoencoder (SAE) model
input_layer = Input(shape=(x_train.shape[1],))
encoder = Dense(32, activation='relu')(input_layer)
encoder = Dense(16, activation='relu')(encoder)
decoder = Dense(32, activation='relu')(encoder)
decoder = Dense(x_train.shape[1], activation='sigmoid')(decoder)

sae = Model(inputs=input_layer, outputs=decoder)

In [8]:
# Train the SAE model
sae.compile(optimizer='adam', loss='binary_crossentropy')
history = sae.fit(x_train, x_train, epochs=50, batch_size=16, validation_data=(x_test, x_test))

Epoch 1/50
132/132 [==============================] - 1s 3ms/step - loss: 0.4201 - val_loss: 0.1793
Epoch 2/50
132/132 [==============================] - 0s 947us/step - loss: 0.1579 - val_loss: 0.1456
Epoch 3/50
132/132 [==============================] - 0s 954us/step - loss: 0.1391 - val_loss: 0.1350
Epoch 4/50
132/132 [==============================] - 0s 939us/step - loss: 0.1308 - val_loss: 0.1290
Epoch 5/50
132/132 [==============================] - 0s 939us/step - loss: 0.1253 - val_loss: 0.1244
Epoch 6/50
132/132 [==============================] - 0s 932us/step - loss: 0.1211 - val_loss: 0.1215
Epoch 7/50
132/132 [==============================] - 0s 924us/step - loss: 0.1189 - val_loss: 0.1196
Epoch 8/50
132/132 [==============================] - 0s 939us/step - loss: 0.1177 - val_loss: 0.1186
Epoch 9/50
132/132 [==============================] - 0s 939us/step - loss: 0.1168 - val_loss: 0.1174
Epoch 10/50
132/132 [==============================] - 0s 954us/step - loss: 0.1161 

In [9]:
# Extract the encoder part of the SAE model as a new model
encoder_model = Model(inputs=input_layer, outputs=encoder)

In [10]:
# Encode the train and test sets using the encoder model
x_train_encoded = encoder_model.predict(x_train)
x_test_encoded = encoder_model.predict(x_test)

29/29 [==============================] - 0s 500us/step


In [11]:
# Train and test a logistic regression classifier using the encoded data
from sklearn.linear_model import LogisticRegression

In [12]:
clf = LogisticRegression(random_state=0, max_iter=1000)
clf.fit(x_train_encoded, y_train)
y_pred = clf.predict(x_test_encoded)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score

clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

# Print predicted labels
print(y_pred)
#Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
print('METRICS FOR THE MODEL')
print('Accuracy:', accuracy)
print('Precision:', precision)
print('F1-score:', f1)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

D:\Users\Equipo-PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
